In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 26.7MB/s 
     |████████████████████████████████| 3.3MB 43.7MB/s 
     |████████████████████████████████| 901kB 52.7MB/s 


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random
import numpy as np
from torchtext.legacy import data
SEED = 1234
from tqdm import tqdm
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#%%
dataset=pd.read_excel('/content/drive/MyDrive/hotel.xlsx')
dataset.columns
dataset=dataset[['Rating','Review']]
for i in range(len(dataset)):
  if dataset.Rating[i]>3:
    dataset.Rating[i]='pos'
  elif dataset.Rating[i]<3:
    dataset.Rating[i]='neg'
  else:
    dataset.Rating[i]='ntr'

df=dataset.sort_values('Rating')

df1,df2=df[:20000],df[-20001:-1]

my_data=result = pd.concat([df1,df2])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
print(sum(my_data.Rating=='pos'))
print(sum(my_data.Rating=='neg'))
print(sum(my_data.Rating=='ntr'))

20000
20000
0


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(tokenizer.vocab)

tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

['hello', 'world', 'how', 'are', 'you', '?']
[7592, 2088, 2129, 2024, 2017, 1029]
[CLS] [SEP] [PAD] [UNK]
101 102 0 100


In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

101 102 0 100
512


In [ ]:
my_data.to_csv('my_data.csv', index=False)

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

my_dataset= data.TabularDataset(
    path='my_data.csv',
    format='csv',
    skip_header=True,
    fields=[('Rating',LABEL),('Review',TEXT)])

In [ ]:
train_data, test_data = my_dataset.split(random_state = random.seed(SEED))

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

print(vars(train_data.examples[6]))


tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['Review'])

print(tokens)

LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.BucketIterator(
    (train_data),
    batch_size = BATCH_SIZE,
    device = device)
valid_iterator= data.BucketIterator(
    (valid_data),
    batch_size = BATCH_SIZE,
    device = device)
test_iterator = data.BucketIterator(
    (test_data),
    batch_size = BATCH_SIZE,
    device = device)

from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

Number of training examples: 19600
Number of validation examples: 8400
Number of testing examples: 12000
{'Rating': 'pos', 'Review': [2023, 4825, 2003, 1996, 3819, 1057, 9333, 7987, 4609, 2818, 3962, 1012, 1996, 7254, 10747, 2003, 2307, 1012, 1045, 3641, 1996, 4880, 21202, 2072, 2029, 2003, 4850, 2606, 24857, 2007, 27940, 1010, 18414, 13344, 20130, 1998, 2712, 8040, 8095, 11923, 1012, 1996, 18414, 13344, 20130, 1010, 27940, 1998, 2712, 8040, 8095, 11923, 2020, 21305, 2007, 14894, 999, 2061, 12090, 1045, 1005, 1049, 26308, 3436, 2004, 1045, 1005, 1049, 22868, 2021, 1996, 24857, 2001, 5410, 1012, 2009, 1005, 1055, 7463, 1999, 1037, 27940, 22953, 2705, 2021, 1045, 2371, 2009, 2734, 11968, 7834, 2319, 2000, 5587, 1037, 2210, 5926, 1012, 1012, 1012, 1037, 2210, 2242, 1012, 2005, 18064, 1045, 3641, 1996, 14418, 10199, 9850, 1010, 2009, 2001, 2074, 2205, 4086, 2005, 2026, 16663, 1012, 2673, 2182, 2001, 2204, 1010, 1998, 1045, 2763, 2097, 2022, 2746, 2067, 2043, 1996, 19350, 10747, 7480, 1012,

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from tqdm import tqdm
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):

        super().__init__()

        self.bert = bert

        embedding_dim = bert.config.to_dict()['hidden_size']

        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          batch_first=True,
                          dropout=0 if n_layers < 2 else dropout)

        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text = [batch size, sent len]

        with torch.no_grad():
            embedded = self.bert(text)[0]

        # embedded = [batch size, sent len, emb dim]

        _, hidden = self.rnn(embedded)

        # hidden = [n layers * n directions, batch size, emb dim]

        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        else:
            hidden = self.dropout(hidden[-1, :, :])

        # hidden = [batch size, hid dim]

        output = self.out(hidden)

        # output = [batch size, out dim]

        return output


HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False



def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in tqdm(iterator):
        optimizer.zero_grad()

        predictions = model(batch.Review).squeeze(1)

        loss = criterion(predictions, batch.Rating)

        acc = binary_accuracy(predictions, batch.Rating)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in tqdm(iterator):
            predictions = model(batch.Review).squeeze(1)

            loss = criterion(predictions, batch.Rating)

            acc = binary_accuracy(predictions, batch.Rating)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

The model has 112,241,409 trainable parameters
The model has 2,759,169 trainable parameters
rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

  0%|          | 0/154 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 22m 11s
	Train Loss: 0.173 | Train Acc: 93.30%
	 Val. Loss: 0.073 |  Val. Acc: 97.50%


  0%|          | 0/154 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 22m 11s
	Train Loss: 0.085 | Train Acc: 96.88%
	 Val. Loss: 0.070 |  Val. Acc: 97.57%


  0%|          | 0/154 [00:00<?, ?it/s]

Epoch: 03 | Epoch Time: 22m 9s
	Train Loss: 0.062 | Train Acc: 97.79%
	 Val. Loss: 0.086 |  Val. Acc: 97.35%


  0%|          | 0/154 [00:00<?, ?it/s]

Epoch: 04 | Epoch Time: 22m 11s
	Train Loss: 0.055 | Train Acc: 98.11%
	 Val. Loss: 0.086 |  Val. Acc: 96.69%


100%|██████████| 66/66 [06:14<00:00,  5.67s/it]


Epoch: 05 | Epoch Time: 22m 11s
	Train Loss: 0.044 | Train Acc: 98.41%
	 Val. Loss: 0.056 |  Val. Acc: 98.04%


In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "This hotel is terrible")